<a href="https://colab.research.google.com/github/Quddos/machine-learning/blob/main/AP24135010028.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Q1:** A program to compute the mean and variance of the elements present in the array.

In [2]:
import numpy as np

# Step 1: Create a 1xN vector (array)
X = np.array([2, 4, 6, 8, 10])  # Example vector

# Step 2: Compute mean
mean_X = np.mean(X)

# Step 3: Compute variance
variance_X = np.var(X)

# Step 4: Print results
print("Vector X:", X)
print("Mean of X:", mean_X)
print("Variance of X:", variance_X)


Vector X: [ 2  4  6  8 10]
Mean of X: 6.0
Variance of X: 8.0


In [3]:
import numpy as np

# Step 1: Define vectors
X = np.array([2, 4, 6, 8, 10])
Y = np.array([1, 3, 5, 7, 9])

# Step 2: Compute covariance manually
mean_X = np.mean(X)
mean_Y = np.mean(Y)
cov_manual = np.sum((X - mean_X) * (Y - mean_Y)) / len(X)

# Step 3: Compute covariance using NumPy
cov_matrix = np.cov(X, Y, bias=True)  # bias=True for population covariance
cov_numpy = cov_matrix[0, 1]

# Step 4: Print results
print("Vector X:", X)
print("Vector Y:", Y)
print("Mean of X:", mean_X, "Mean of Y:", mean_Y)
print("Covariance (Manual):", cov_manual)
print("Covariance (NumPy):", cov_numpy)


Vector X: [ 2  4  6  8 10]
Vector Y: [1 3 5 7 9]
Mean of X: 6.0 Mean of Y: 5.0
Covariance (Manual): 8.0
Covariance (NumPy): 8.0
